![pokemon](https://people.emich.edu/jhe5/fall16COSC231/project1/banner.jpg)

# Creating a Linear Regression with Data from the Graph

Now we're going to use the data we've collected and the graphs we've created to make a simple visual: a linear regression.

## 1.0 Connect to the Solution
We'll first install and import pyTigerGraph then create our TigerGraphConnection.

In [ ]:
!pip install pyTigerGraph
import pyTigerGraph as tg

conn = tg.TigerGraphConnection(host="https://pokemon.i.tgcloud.io", password="tigergraph", graphname="PokemonGraph")
conn.apiToken = conn.getToken(conn.createSecret())

## 2.0 Write the Query
Next, we'll create a getAttributes query. This contains several accumulators which will grab the heights, weights, names, and types of the Pokémon. 

In [ ]:
print(conn.gsql('''
DROP QUERY getAttributes
CREATE QUERY getAttributes() FOR GRAPH PokemonGraph { 
  ListAccum<INT> @@heights, @@weights;
  ListAccum<STRING> @@names, @@types;
  SetAccum<STRING> @@types_set;
  
  catchThemAll = {Pokemon.*};
  
  Res = SELECT t FROM catchThemAll:p-(POKEMON_TYPE:e)-Type:t 
        ACCUM @@heights+=p.height, @@weights+=p.weight,
              @@names+=p.name, @@types+=t.type_id,
              @@types_set+=t.type_id;
  
  PRINT @@heights, @@weights, @@names, @@types, @@types_set;
}
INSTALL QUERY getAttributes
'''))

[                                                                                     ] 0% (0/1)
[                                                                                     ] 0% (0/1)
[========================================                                             ] 47% (0/1)
[=================================================================================    ] 95% (0/1)
[=====================================================================================] 100% (1/1)

The query getAttributes is dropped.
The query getAttributes has been added!
Start installing queries, about 1 minute ...
getAttributes query: curl -X GET 'https://127.0.0.1:9000/query/PokemonGraph/getAttributes'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.



## 3.0 Plot it!
Finally, we'll plot it all out!

In [ ]:
import plotly.express as px
import pandas as pd

att = conn.runInstalledQuery("getAttributes")[0]

heights, weights, types, names = att["@@heights"], att["@@weights"], att["@@types"], att["@@names"]
colour_map = {val : num for num, val in (enumerate(att["@@types_set"]))}
colours = [colour_map[i]/len(att["@@types_set"]) for i in types]

d = {'height': heights, 'weight': weights, 'type': types, 'name': names, 'type_by_colour': colours}
df = pd.DataFrame(data=d)
fig = px.scatter(df, x="height", y="weight", color="type_by_colour", title="Pokemon by Height/Weight and Types", hover_name=names, hover_data=['name', 'type'], trendline="ols")
fig.show()


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
import plotly.express as px
import pandas as pd

att = conn.runInstalledQuery("getAttributes")[0]

heights, weights, types, names = att["@@heights"], att["@@weights"], att["@@types"], att["@@names"]
colour_map = {val : num for num, val in (enumerate(att["@@types_set"]))}
colours = [colour_map[i]/len(att["@@types_set"]) for i in types]

d = {'height': heights, 'weight': weights, 'type': types, 'name': names, 'type_by_colour': colours}
df = pd.DataFrame(data=d)
fig = px.scatter(df, x="height", y="weight", color="type_by_colour", title="Pokemon by Height/Weight and Types", hover_name=names, hover_data=['name', 'type'], trendline="ols")
# fig.show()

results = px.get_trendline_results(fig)
print(results)

# results.query("type == 'fire' and type == 'water'").px_fit_results.iloc[0].summary()


                                      px_fit_results
0  <statsmodels.regression.linear_model.Regressio...


## 4.0 Congrats!

Wow! This is the final lesson of this course (for now!). I hope you continue exploring TigerGraph and all of the other use cases! 

* If you want to talk with other developers, please join the Discord: https://discord.gg/F2c9b9v
* If you want to explore more TigerGraph tools, please check out the developer portal: https://developers.tigergraph.com/
